### instalação das bibliotecas necessarias

In [ ]:
%pip install pandas
%pip install numpy 

### importações das bibliotecas instaladas

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [3]:
df_netflix = pd.read_csv("C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados/data_netflix.csv")

In [88]:
df_netflix

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Ariel,movie,"Comedy, Crime, Romance",1988.0,tt0094675,7.4,9188.0,NaN
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986.0,tt0092149,7.4,8014.0,NaN
2,Forrest Gump,movie,"Drama, Romance",1994.0,tt0109830,8.8,2381047.0,NaN
3,American Beauty,movie,Drama,1999.0,tt0169547,8.3,1249470.0,NaN
4,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,527520.0,NaN
...,...,...,...,...,...,...,...,...
21811,NaN,tv,NaN,2009.0,NaN,NaN,NaN,NaN
21812,Cheat: Unfinished Business,tv,Reality-TV,2025.0,tt32249973,5.0,191.0,NaN
21813,NaN,tv,"Animation, Crime, Mystery",1996.0,NaN,NaN,NaN,NaN
21814,NaN,tv,NaN,2014.0,NaN,NaN,NaN,NaN


In [56]:
df_netflix.describe()

,releaseYear,imdbAverageRating,imdbNumVotes
count,21776.000000,19987.000000,1.998700e+04
mean,2013.358927,6.400070,3.226471e+04
std,14.310762,1.092932,1.219602e+05
min,1913.000000,1.000000,5.000000e+00
25%,2011.000000,5.700000,3.300000e+02
50%,2018.000000,6.500000,1.616000e+03
75%,2022.000000,7.200000,1.042050e+04
max,2025.000000,9.800000,3.046940e+06


In [57]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21816 entries, 0 to 21815
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               21085 non-null  object 
 1   type                21816 non-null  object 
 2   genres              21444 non-null  object 
 3   releaseYear         21776 non-null  float64
 4   imdbId              20170 non-null  object 
 5   imdbAverageRating   19987 non-null  float64
 6   imdbNumVotes        19987 non-null  float64
 7   availableCountries  158 non-null    object 
dtypes: float64(3), object(5)
memory usage: 1.3+ MB


In [4]:
df_netflix.isnull().sum()

title                   731
type                      0
genres                  372
releaseYear              40
imdbId                 1646
imdbAverageRating      1829
imdbNumVotes           1829
availableCountries    21658
dtype: int64

#### =======excluindo linhas sem titulo===========

In [5]:
df_netflix.dropna(subset=['title'], inplace=True)

#### ========tratamento de nulos de genero =============

In [6]:
#preencher os NaN de genres com "Não informado"
df_netflix['genres'] = df_netflix['genres'].fillna('Não informado')

### =====tratamento de nulos de imdb =======

In [7]:
# os NaN em imdbId e imdbAverageRating podem ser um problema se for analisar a avaliação dos filmes, então preciso ver se o NaN em um coincide com NaN na outra

# Investigar a situação dos NaN
print("Registros sem imdbId:", df_netflix['imdbId'].isna().sum())
print("Registros sem rating:", df_netflix['imdbAverageRating'].isna().sum())

# Ver se NaN em imdbId implica em NaN em rating
mask_sem_id = df_netflix['imdbId'].isna()
print("Dos que não têm ID, quantos também não têm rating:", 
      df_netflix[mask_sem_id]['imdbAverageRating'].isna().sum())

Registros sem imdbId: 915
Registros sem rating: 1098
Dos que não têm ID, quantos também não têm rating: 915


In [8]:
# Investigar os casos com ID mas sem avaliação
tem_id_sem_rating = df_netflix['imdbId'].notna() & df_netflix['imdbAverageRating'].isna()
print("Registros com ID mas sem avaliação:", tem_id_sem_rating.sum())

# Ver alguns exemplos desses casos
print("\nExemplos de conteúdo com ID mas sem avaliação:")
exemplos = df_netflix[tem_id_sem_rating][['title', 'imdbId', 'releaseYear']].head()
print(exemplos)

Registros com ID mas sem avaliação: 183

Exemplos de conteúdo com ID mas sem avaliação:
                                         title      imdbId  releaseYear
3048                          Queer My Friends  tt28626426       2023.0
3135                            Adel Mesh Adel  tt36596034       2024.0
3164                                   Katinka  tt14281786       2017.0
4711  Ruy, el caballero de la espada de madera  tt27518740       2002.0
6258                       De Fil en Aiguilles   tt3613444       2013.0


In [9]:
#preencher os NaN de imdbid com "Não informado"
df_netflix['imdbId'] = df_netflix['imdbId'].fillna('Não informado')

#### Não faz sentido que um filme que não foi avaliado tenha media de avaliações, então a correlação que sobra é de avaliação com media de avaliação, se tirar os nulos de avaliação, somem os da media de avaliação

#### Usando a mediana da avaliação dos filmes de um determinado gênero pra preencher os NaN da avaliação no imdb 

A função impute_ratings_by_genre_decade(df_netflix) serve para preencher (imputar) valores ausentes (NaN) na coluna de avaliações médias do IMDb (imdbAverageRating) do DataFrame da Netflix, usando a mediana das avaliações de filmes do mesmo gênero e da mesma década de lançamento.

* Passos principais da função:
    * Cópia do DataFrame: Cria uma cópia do DataFrame original para não alterar os dados originais.

    * Gênero primário: Extrai o primeiro gênero listado em genres para cada filme       (considerando que um filme pode ter vários gêneros separados por vírgula).

    * Década: Calcula a década de lançamento a partir do ano (releaseYear), por exemplo, 1995 vira 1990.

    * Flag de rating original: Cria uma coluna indicando se o filme já tinha avaliação (imdbAverageRating) ou não.

    * Mediana por gênero e década: Calcula a mediana das avaliações para cada combinação de gênero primário e década, usando apenas os filmes que já têm avaliação.

    * Fallbacks: Se não houver mediana para uma combinação específica, usa a mediana do gênero. Se ainda assim não houver, usa a mediana global de todos os filmes avaliados.

* Objetivo: Preencher os valores ausentes de avaliação de forma mais contextualizada, levando em conta o gênero e a época do filme, tornando a análise posterior mais robusta e realista.

In [ ]:
def impute_ratings_by_genre_decade(df):
    """
    Imputa valores NaN de imdbAverageRating usando mediana por gênero primário + década
    
    Args:
        df: DataFrame com colunas 'genres', 'releaseYear', 'imdbAverageRating'
    
    Returns:
        DataFrame com ratings imputados e colunas adicionais para rastreamento
    """
    # Extrair gênero primário (primeiro da lista)
    df['primaryGenre'] = df['genres'].fillna('Unknown').str.split(',').str[0].str.strip()

    # Extrair década
    df['decade'] = (df['releaseYear'] // 10) * 10

    # Flag para indicar se tinha rating original
    df['hasOriginalRating'] = df['imdbAverageRating'].notna()

    # Calcular medianas por gênero + década (apenas filmes COM rating)
    films_with_rating = df[df['hasOriginalRating']]
    
    medians_genre_decade = (films_with_rating.groupby(['primaryGenre', 'decade'])['imdbAverageRating'].agg(['median', 'count']).reset_index())

    # Calcular medianas só por gênero (fallback)
    medians_genre = (films_with_rating.groupby('primaryGenre')['imdbAverageRating'].median().reset_index())
    
    # Mediana global (último recurso)
    global_median = films_with_rating['imdbAverageRating'].median()

    # Função aninhada para imputar um filme específico
    def impute_single_film(row):
        if row['hasOriginalRating']:
            return pd.Series({
                'imdbAverageRating': row['imdbAverageRating'],
                'ratingImputed': False,
                'imputationSource': 'original'
            })
        
        # Tentar gênero + década primeiro
        genre_decade_match = medians_genre_decade[
            (medians_genre_decade['primaryGenre'] == row['primaryGenre']) & 
            (medians_genre_decade['decade'] == row['decade'])
        ]
        
        if not genre_decade_match.empty:
            median_val = genre_decade_match.iloc[0]['median']
            count = genre_decade_match.iloc[0]['count']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} {int(row['decade'])}s ({count} filmes)"
            })
        
        # Fallback: só gênero
        genre_match = medians_genre[medians_genre['primaryGenre'] == row['primaryGenre']]
        
        if not genre_match.empty:
            median_val = genre_match.iloc[0]['imdbAverageRating']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} geral"
            })
        
        # Último recurso: mediana global
        return pd.Series({
            'imdbAverageRating': global_median,
            'ratingImputed': True,
            'imputationSource': "mediana global"
        })

    # Aplicar imputação
    imputation_results = df.apply(impute_single_film, axis=1)

    # Combinar resultados
    df['imdbAverageRating'] = imputation_results['imdbAverageRating']
    df['ratingImputed'] = imputation_results['ratingImputed']
    df['imputationSource'] = imputation_results['imputationSource']

    return df, medians_genre_decade, global_median

In [11]:
# APLICAR A IMPUTAÇÃO
df_imputed, medians_table, global_median = impute_ratings_by_genre_decade(df_netflix)

In [12]:
imputed_count = df_imputed['ratingImputed'].sum()
print(f"Valores imputados: {imputed_count}")

Valores imputados: 1098


In [13]:
# Mostrar alguns exemplos organizados
for i, (_, filme) in enumerate(df_imputed.head(8).iterrows()):
    print(f"{i+1:2d}. \"{filme['title']}\" ({int(filme['releaseYear'])})")
    print(f"    Gênero: {filme['primaryGenre']}")
    print(f"    Avaliação imputada: {filme['imdbAverageRating']:.1f}")
    print(f"    Fonte: {filme['imputationSource']}")
    print()

 1. "Ariel" (1988)
    Gênero: Comedy
    Avaliação imputada: 7.4
    Fonte: original

 2. "Shadows in Paradise" (1986)
    Gênero: Comedy
    Avaliação imputada: 7.4
    Fonte: original

 3. "Forrest Gump" (1994)
    Gênero: Drama
    Avaliação imputada: 8.8
    Fonte: original

 4. "American Beauty" (1999)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 5. "The Fifth Element" (1997)
    Gênero: Action
    Avaliação imputada: 7.6
    Fonte: original

 6. "Jarhead" (2005)
    Gênero: Biography
    Avaliação imputada: 7.0
    Fonte: original

 7. "Unforgiven" (1992)
    Gênero: Drama
    Avaliação imputada: 8.2
    Fonte: original

 8. "Eternal Sunshine of the Spotless Mind" (2004)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original



#### A coluna availableCountries tem NaN na maioria das linhas, significa que a maioria dos titulos não está com seus paises disponiveis descritos, isso pode ser um case especifico mas pra nossa realidade não acredito que seja relevante. 

In [14]:
# Axis 1 para colunas, axis 0 para linhas
df_netflix = df_netflix.drop(['availableCountries'], axis=1)

In [102]:
df_netflix.isnull().sum()

title                   0
type                    0
genres                  0
releaseYear             3
imdbId                  0
imdbAverageRating       0
imdbNumVotes         1098
primaryGenre            0
decade                  3
hasOriginalRating       0
ratingImputed           0
imputationSource        0
dtype: int64

#### vamos ver quem são esses 3 filmes que ainda sobraram entre os nulls em ano de lançamento

In [15]:
# Ver quais são os 3 filmes problemáticos
nulls_year = df_netflix[df_netflix['releaseYear'].isna()]
print(nulls_year[['title', 'type', 'genres', 'imdbId', 'imdbAverageRating']])

                                 title   type             genres  \
12661  Rudra: Secret of the Black Moon  movie      Não informado   
16302       aiko Love Like Aloha vol.7  movie      Não informado   
20685     3-nen Z-gumi Ginpachi Sensei     tv  Animation, Comedy   

              imdbId  imdbAverageRating  
12661  Não informado                8.6  
16302  Não informado                8.6  
20685     tt27182605                7.0  


In [16]:
#### não são filmes significativos e tentar tratar vai acbar poluindo a analise
df_netflix = df_netflix.dropna(subset=['releaseYear'])

In [17]:
#### só depois de remover todos os NaN é que posso converter o releaseYear para int
df_netflix['releaseYear'] = df_netflix['releaseYear'].astype(int)

#### me resta o problema do demais imdbNumVotes ainda NaN, além das colunas extras criadas para o tratamento e que agora não tem mais função

In [19]:
# ==========================================
# LIMPEZA FINAL - REMOVER BAGAGEM TÉCNICA
# ==========================================

# 1. TRATAR imdbNumVotes para filmes imputados
print("🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS")
print("=" * 50)

# Para filmes com rating imputado, vamos usar a mediana de votos do mesmo gênero/década
films_with_votes = df_netflix[df_netflix['imdbNumVotes'].notna()]

# Calcular mediana de votos por gênero primário
median_votes_by_genre = (films_with_votes
                        .groupby('primaryGenre')['imdbNumVotes']
                        .median()
                        .to_dict())

# Mediana global como fallback
global_median_votes = films_with_votes['imdbNumVotes'].median()

# Função para imputar votos
def impute_votes(row):
    if pd.notna(row['imdbNumVotes']):
        return row['imdbNumVotes']
    
    # Se tem rating imputado, usar mediana do gênero
    if row['ratingImputed']:
        genre_median = median_votes_by_genre.get(row['primaryGenre'])
        if genre_median:
            return genre_median
        else:
            return global_median_votes
    
    return row['imdbNumVotes']



🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS


In [20]:
# Aplicar imputação de votos
df_netflix['imdbNumVotes'] = df_netflix.apply(impute_votes, axis=1)


In [21]:
df_netflix.isnull().sum()

title                0
type                 0
genres               0
releaseYear          0
imdbId               0
imdbAverageRating    0
imdbNumVotes         0
primaryGenre         0
decade               0
hasOriginalRating    0
ratingImputed        0
imputationSource     0
dtype: int64

In [22]:
df_netflix['imdbNumVotes'] = df_netflix['imdbNumVotes'].astype(int)
# Verificar se a conversão foi bem-sucedida
print(df_netflix.dtypes)

title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
primaryGenre          object
decade               float64
hasOriginalRating       bool
ratingImputed           bool
imputationSource      object
dtype: object


#### =====Ideia de Tratamento pra ML =======

In [24]:
# ==========================================
# CRIANDO DATA QUALITY SCORE
# ==========================================

def calculate_data_quality_score(df):
    """
    Calcula score de qualidade dos dados (0-1) baseado em campos imputados
    1.0 = dados 100% originais
    0.0 = todos os dados principais foram imputados
    """
    
    # Campos principais para avaliar qualidade
    main_fields = ['title', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes']
    
    # Conta quantos campos foram imputados/tratados por linha
    df['imputed_fields_count'] = 0
    df['total_main_fields'] = len(main_fields)
    
    # Verifica cada campo tratado
    # Title: teve que dropar, não está mais no dataset
    
    # Genres: se era NaN e virou "Não informado"
    df.loc[df['genres'] == 'Não informado', 'imputed_fields_count'] += 1
    
    # ImdbId: se era NaN e virou "Não informado"  
    df.loc[df['imdbId'] == 'Não informado', 'imputed_fields_count'] += 1
    
    # Rating: se foi imputado (você já tem essa flag!)
    df.loc[df['ratingImputed'] == True, 'imputed_fields_count'] += 1
    
    # Votes: se o rating foi imputado, os votes também foram
    df.loc[df['ratingImputed'] == True, 'imputed_fields_count'] += 1
    
    # Calcula o score (inverte para que 1.0 = melhor qualidade)
    df['data_quality_score'] = 1.0 - (df['imputed_fields_count'] / df['total_main_fields'])
    
    # Categoriza a qualidade
    df['quality_category'] = pd.cut(
        df['data_quality_score'], 
        bins=[0, 0.5, 0.8, 1.0], 
        labels=['Low', 'Medium', 'High'],
        include_lowest=True
    )
    
    return df


In [25]:

# Aplicar a função
df_netflix = calculate_data_quality_score(df_netflix)


In [26]:

# ==========================================
# ANÁLISE RÁPIDA DA QUALIDADE
# ==========================================

def analyze_data_quality(df, platform_name):
    """Mostra estatísticas de qualidade dos dados"""
    print(f"\n📊 QUALIDADE DOS DADOS - {platform_name.upper()}")
    print("=" * 50)
    
    # Distribuição por categoria
    quality_dist = df['quality_category'].value_counts()
    total = len(df)
    
    for category, count in quality_dist.items():
        percentage = (count / total) * 100
        print(f"{category} Quality: {count:,} filmes ({percentage:.1f}%)")
    
    # Estatísticas do score
    print(f"\nScore médio: {df['data_quality_score'].mean():.3f}")
    print(f"Score mediano: {df['data_quality_score'].median():.3f}")
    
    # Campos mais problemáticos
    print(f"\nCampos imputados por registro:")
    print(f"  Média: {df['imputed_fields_count'].mean():.2f}")
    print(f"  Máximo: {df['imputed_fields_count'].max()}")
    
    return df



In [27]:
# Analisar Netflix
analyze_data_quality(df_netflix, "Netflix")


📊 QUALIDADE DOS DADOS - NETFLIX
High Quality: 19,987 filmes (94.8%)
Low Quality: 914 filmes (4.3%)
Medium Quality: 181 filmes (0.9%)

Score médio: 0.974
Score mediano: 1.000

Campos imputados por registro:
  Média: 0.16
  Máximo: 4


,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,primaryGenre,decade,hasOriginalRating,ratingImputed,imputationSource,imputed_fields_count,total_main_fields,data_quality_score,quality_category
0,Ariel,movie,"Comedy, Crime, Romance",1988,tt0094675,7.4,9188,Comedy,1980.0,True,False,original,0,6,1.0,High
1,Shadows in Paradise,movie,"Comedy, Drama, Music",1986,tt0092149,7.4,8014,Comedy,1980.0,True,False,original,0,6,1.0,High
2,Forrest Gump,movie,"Drama, Romance",1994,tt0109830,8.8,2381047,Drama,1990.0,True,False,original,0,6,1.0,High
3,American Beauty,movie,Drama,1999,tt0169547,8.3,1249470,Drama,1990.0,True,False,original,0,6,1.0,High
4,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997,tt0119116,7.6,527520,Action,1990.0,True,False,original,0,6,1.0,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21805,Franklin,tv,Drama,2025,tt36279637,4.9,164,Drama,2020.0,True,False,original,0,6,1.0,High
21808,Happy Crush,tv,"Comedy, Romance",2025,tt36587037,8.8,12,Comedy,2020.0,True,False,original,0,6,1.0,High
21809,Fred & Rose West: A British Horror Story,tv,"Crime, Documentary",2025,tt36590949,6.5,1642,Crime,2020.0,True,False,original,0,6,1.0,High
21812,Cheat: Unfinished Business,tv,Reality-TV,2025,tt32249973,5.0,191,Reality-TV,2020.0,True,False,original,0,6,1.0,High


In [28]:
# CRIAR DATASET LIMPO (apenas colunas essenciais) e removendo as colunas criadas para o tratamento
print(f"\n📂 CRIANDO DATASET LIMPO")
print("=" * 30)

colunas_essenciais = [
    'title', 
    'type', 
    'genres', 
    'releaseYear', 
    'imdbId', 
    'imdbAverageRating', 
    'imdbNumVotes'
]

df_netflix_clean = df_netflix[colunas_essenciais].copy()

print(f"Forma final do dataset: {df_netflix_clean.shape}")
print(f"Colunas finais: {list(df_netflix_clean.columns)}")


📂 CRIANDO DATASET LIMPO
Forma final do dataset: (21082, 7)
Colunas finais: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes']


In [ ]:
# ==========================================
# ATUALIZAR DATASET LIMPO
# ==========================================

# Adicionar as colunas de qualidade ao dataset final
colunas_essenciaisML = [
    'title', 
    'type', 
    'genres', 
    'releaseYear', 
    'imdbId', 
    'imdbAverageRating', 
    'imdbNumVotes',
    'data_quality_score',
    'quality_category'
]

# Exemplo para Netflix (repita para outros)
df_netflix_cleanML = df_netflix[colunas_essenciaisML].copy()
print(f"\n✅ Dataset NetflixML: {df_netflix_cleanML.shape}")
print(f"Colunas finais: {list(df_netflix_cleanML.columns)}")

# Salvar versão atualizada
df_netflix_cleanML.to_csv("data_netflix_cleanML.csv", index=False)


✅ Dataset NetflixML: (21082, 9)
Colunas finais: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes', 'data_quality_score', 'quality_category']


In [116]:
# VERIFICAÇÃO FINAL
print(f"\n✅ VERIFICAÇÃO FINAL")
print("=" * 35)

print("Valores nulos por coluna:")
print(df_netflix_clean.isnull().sum())

print(f"\nTipos de dados:")
print(df_netflix_clean.dtypes)


✅ VERIFICAÇÃO FINAL
Valores nulos por coluna:
title                0
type                 0
genres               0
releaseYear          0
imdbId               0
imdbAverageRating    0
imdbNumVotes         0
dtype: int64

Tipos de dados:
title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
dtype: object


### ========== Exploração tipo e genero ================

In [129]:
tipos_unicos = df_netflix_clean['type'].unique()
print("Tipos únicos na coluna 'type':", tipos_unicos)

Tipos únicos na coluna 'type': ['movie' 'tv']


In [ ]:
# Contando quantas produções existem de cada tipo
contagem_tipos = df_netflix_clean['type'].value_counts()
print(f"\nDistribuição por tipo:")
for tipo, quantidade in contagem_tipos.items():
    percentual = (quantidade / len(df_netflix_clean)) * 100
    print(f"{tipo}: {quantidade} produções ({percentual:.1f}%)")


Distribuição por tipo:
movie: 16473 produções (78.1%)
tv: 4609 produções (21.9%)


In [ ]:
# EXPLORANDO OS PADRÕES DOS GENRES
print("\nPADRÕES E ESTRUTURA DOS GENRES")
# Amostras aleatórias para entender o formato
print("Exemplos de como os genres aparecem:")
sample_genres = df_netflix_clean['genres'].dropna().sample(10).values
for i, genre in enumerate(sample_genres, 1):
    print(f"{i:2d}. {genre}")


PADRÕES E ESTRUTURA DOS GENRES
Exemplos de como os genres aparecem:
 1. Documentary
 2. Action, Comedy, Thriller
 3. Horror, Short
 4. Comedy, Musical, Romance
 5. Action
 6. Action, Sci-Fi, Thriller
 7. Action, Thriller
 8. Action, Adventure, Drama
 9. Animation, Comedy, Fantasy
10. Drama, History, War


In [ ]:
# Função para extrair genres individuais
def extract_individual_genres(df_netflix_clean):
    """Extrai todos os genres individuais de uma coluna que pode conter múltiplos genres"""
    all_genres = []
    
    for genres_str in df_netflix_clean['genres'].dropna():
        # Remove espaços e divide por vírgulas (assumindo que é o separador mais comum)
        if pd.notna(genres_str):
            # Tratamento flexível para diferentes separadores
            genres_list = re.split(r'[,;|]', str(genres_str))
            # Limpa e adiciona à lista
            clean_genres = [genre.strip() for genre in genres_list if genre.strip()]
            all_genres.extend(clean_genres)
    
    return all_genres

#### a biblioteca re permite realizar operações com operações regulares, evitabndo todo o trabalho de digitar algo como normalized = str(genres_str).replace(';', ',').replace('|', ',')
#### genres_list = normalized.split(',') toda vez que precisarmos separar strings

In [ ]:
# Extraindo todos os genres individuais
individual_genres = extract_individual_genres(df_netflix_clean)

# Com Counter é a mesma coisa, ele evita usar laços longos como genre_counts = {}
for genre in individual_genres:
    if genre in genre_counts:
        genre_counts[genre] += 1
    else:
        genre_counts[genre] = 1

de forma que só é preciso importar a biblioteca e chama-la para percorre toda a lista de gêneros e ele mantém um registro automático de quantas vezes cada gênero único aparece

In [122]:
# Contando frequencias
genre_counts = Counter(individual_genres)
print(f"Total de genres únicos encontrados: {len(genre_counts)}")
print(f"Total de ocorrências de genres: {sum(genre_counts.values())}")

Total de genres únicos encontrados: 33
Total de ocorrências de genres: 45922


In [123]:
# TOP GENRES MAIS POPULARES
print("\n TOP 20 GENRES MAIS POPULARES")
top_genres = genre_counts.most_common(20)
for i, (genre, count) in enumerate(top_genres, 1):
    percentage = (count / sum(genre_counts.values())) * 100
    print(f"{i:2d}. {genre:<25} | {count:>6} ocorrências ({percentage:>5.2f}%)")



 TOP 20 GENRES MAIS POPULARES
 1. Drama                     |   9879 ocorrências (21.51%)
 2. Comedy                    |   7528 ocorrências (16.39%)
 3. Action                    |   3519 ocorrências ( 7.66%)
 4. Crime                     |   3088 ocorrências ( 6.72%)
 5. Romance                   |   3083 ocorrências ( 6.71%)
 6. Adventure                 |   2514 ocorrências ( 5.47%)
 7. Documentary               |   2251 ocorrências ( 4.90%)
 8. Thriller                  |   2128 ocorrências ( 4.63%)
 9. Animation                 |   2104 ocorrências ( 4.58%)
10. Mystery                   |   1453 ocorrências ( 3.16%)
11. Horror                    |   1300 ocorrências ( 2.83%)
12. Family                    |   1136 ocorrências ( 2.47%)
13. Fantasy                   |   1038 ocorrências ( 2.26%)
14. Biography                 |    983 ocorrências ( 2.14%)
15. History                   |    758 ocorrências ( 1.65%)
16. Music                     |    627 ocorrências ( 1.37%)
17. Sci-F

In [124]:
# Salva o DataFrame limpo
df_netflix_clean.to_csv("data_netflix_clean.csv", index=False)